In [ ]:
# !wget -O dataset_colors.zip "https://downloader.disk.yandex.ru/disk/bbf64f7a168a28e1d1d26070680e49ec208b557df29838280b5ef59ceec92465/67d3bfbb/WYSj2tn0xwfH9xrDn6OMB8Xma9PEMVVkSdj-oTw_no6Lf3VlXmTFMBrR9uEtahv-HyUjYUqUXYJgg46saw6Mxw%3D%3D?uid=0&filename=dataset_colors.zip&disposition=attachment&hash=Xb8qDbUG6BngPcy%2B0ZCrTUCidjMJdvUIof229dcBskZkF3VkCsI1pAlXMCohgSKXq/J6bpmRyOJonT3VoXnDag%3D%3D%3A/dataset_colors.zip&limit=0&content_type=application%2Fzip&owner_uid=957907626&fsize=9820116123&hid=a728aa4ce5f261d0a30ab7a0cf8b38fb&media_type=compressed&tknv=v2"

In [ ]:
# !unzip /content/dataset_colors.zip

In [ ]:
from transformers import AutoModel, AutoProcessor
import torch
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [ ]:
df = pd.read_csv('/kaggle/input/avitohack/dataset_colors/train_data.csv')

categories_dict = {"сумки": "bags", "столы": "tables", "одежда для девочек": "clothings", "стулья": "chairs"}
colors_dict = {"zelenyi": "green", "chernyi": "black", "belyi": "white", "bordovyi": "burgundy", "krasnyi": "red", "bezhevyi": "beige", "raznocvetnyi": "multicolored", "rozovyi": "pink", "serebristyi": "silver", "korichnevyi": "brown", "fioletovyi": "purple", "seryi": "gray", "goluboi": "light blue", "oranzhevyi": "orange", "sinii": "blue", "biryuzovyi": "turquoise", "zolotoi": "gold", "zheltyi": "yellow"}

df['text'] = [f'a photo of {i[2]} {i[1]}' for i in df.values]
df['path'] = [f'/kaggle/input/avitohack/dataset_colors/train_data/{i[0]}.jpg' for i in df.values]

df

,id,category,target,text,path
0,11624739720,сумки,zelenyi,a photo of zelenyi сумки,/kaggle/input/avitohack/dataset_colors/train_d...
1,11967354953,сумки,chernyi,a photo of chernyi сумки,/kaggle/input/avitohack/dataset_colors/train_d...
2,19026830493,сумки,belyi,a photo of belyi сумки,/kaggle/input/avitohack/dataset_colors/train_d...
3,23088722867,сумки,chernyi,a photo of chernyi сумки,/kaggle/input/avitohack/dataset_colors/train_d...
4,23546690270,столы,belyi,a photo of belyi столы,/kaggle/input/avitohack/dataset_colors/train_d...
...,...,...,...,...,...
33298,37599243178,одежда для девочек,raznocvetnyi,a photo of raznocvetnyi одежда для девочек,/kaggle/input/avitohack/dataset_colors/train_d...
33299,37599258032,сумки,bezhevyi,a photo of bezhevyi сумки,/kaggle/input/avitohack/dataset_colors/train_d...
33300,37599262080,одежда для девочек,bezhevyi,a photo of bezhevyi одежда для девочек,/kaggle/input/avitohack/dataset_colors/train_d...
33301,37599266008,одежда для девочек,bezhevyi,a photo of bezhevyi одежда для девочек,/kaggle/input/avitohack/dataset_colors/train_d...


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel

model_name = 'jinaai/jina-clip-v1'
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# for param in model.parameters():
#     param.requires_grad = False

# for param in model.text_model.encoder.layers[-2:].parameters():
#     param.requires_grad = True
# for param in model.vision_model.encoder.layers[-2:].parameters():
#     param.requires_grad = True



config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

configuration_clip.py:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-clip-implementation:
- configuration_clip.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_clip.py:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

transform.py:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

JinaCLIPModel(
  (text_model): HFTextEncoder(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30528, 768, padding_idx=0)
        (token_type_embeddings): Embedding(2, 768)
      )
      (emb_drop): Dropout(p=0.1, inplace=False)
      (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (encoder): BertEncoder(
        (layers): ModuleList(
          (0-11): 12 x Block(
            (mixer): MHA(
              (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
              (inner_attn): SelfAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (inner_cross_attn): CrossAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (dropout1): Dropout(p=0.1, inplace=False)
            (drop_path1): StochasticDepth(p=0.0, mode=row)
        

In [ ]:
from sklearn.model_selection import train_test_split

_, train = train_test_split(df, test_size=0.2, stratify=df['category']+df['target'])

In [ ]:
train.target.value_counts()

target
chernyi         781
belyi           416
korichnevyi     367
raznocvetnyi    321
bezhevyi        273
sinii           256
rozovyi         211
seryi           197
zelenyi         116
krasnyi          93
goluboi          83
fioletovyi       53
bordovyi         51
zheltyi          41
oranzhevyi       33
serebristyi      21
zolotoi          15
biryuzovyi        3
Name: count, dtype: int64

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        self.image_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        image = Image.open(item['path']).convert('RGB')
        image = self.image_transform(image)
        text = item['text']
        return {'image': image, 'text': text}

dataset = CustomDataset(train)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)



In [ ]:
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)



In [ ]:
device

'cuda'

In [ ]:
from tqdm import tqdm

model.train()
for epoch in range(5):
    print(f"Epoch {epoch + 1}/{5}")

    pbar = tqdm(dataloader, desc="Training", unit="batch")
    for batch in pbar:
        images, texts = [transforms.ToPILImage()(image) for image in batch['image']], batch['text']

        inputs = processor(
            text=texts,
            images=images,
            return_tensors="pt",
            padding=True,
            truncation=True,
        ).to(device)

        outputs = model(**inputs)

        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(len(images), device=device)
        loss = (F.cross_entropy(logits_per_image, labels) +
               F.cross_entropy(logits_per_text, labels)) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1/5


Training: 100%|██████████| 67/67 [03:09<00:00,  2.82s/batch, loss=1.66]


Epoch 1, Loss: 1.6613121032714844
Epoch 2/5


Training: 100%|██████████| 67/67 [02:21<00:00,  2.12s/batch, loss=1.19]


Epoch 2, Loss: 1.18839693069458
Epoch 3/5


...

Epoch 3, Loss: 0.9564576148986816
Epoch 4/5


Training: 100%|██████████| 67/67 [02:19<00:00,  2.09s/batch, loss=1.21]


Epoch 4, Loss: 1.2135322093963623
Epoch 5/5


Training: 100%|██████████| 67/67 [02:20<00:00,  2.09s/batch, loss=1.16] 

Epoch 5, Loss: 1.1579777002334595


In [ ]:
torch.save(model.state_dict(), "winwin.pth")